Roll : 21226

Assignment 4 : Building NeuralNet

---

## Building Neural Network

Neural networks comprise of layers/modules that perform operations on data. The `torch.nn` namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the `nn.Module`.

A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# Selecting Device for training

device = (
    'cuda' if torch.cuda.is_available()
    else 'mps' if torch.backends.mps.is_available()
    else 'cpu'
)
print('Device being used : ', device)

Device being used :  mps


## Defining Neural Net Class

Here, we define the neural network and its layers.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device) # Sending model compute to whichever device is available

print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


**We must not call `model.forward()` directly since there are some background operations when passing it with input data!!**

When we call the model on input, it returns a 2D Tensor with `dim = 0`, corresponding to each output of 10 predicted values of every class and `dim = 1` corresponding to the individual value of the output.

We then use `nn.Softmax` to get prediction probabilities.

In [4]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8], device='mps:0')


---

## Breaking down Model Layers

Input:

In [5]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### `nn.Flatten`

Converts 2D array to 1D 784 pixel values.

In [6]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### `nn.Linear`

Linear Transformation is applied on input using stored weights and biases.

In [7]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### `nn.ReLU`

Non-Linearity helps create Complex Mappings between the model inputs and outputs.

In [8]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 3.2033e-01,  3.7399e-02,  4.4564e-01, -1.3831e-01,  6.7761e-01,
         -1.2061e-01,  1.4613e-01, -1.1622e-01,  1.4939e-01,  8.3715e-02,
          4.7948e-01, -2.6636e-01,  3.3539e-01,  5.6041e-01, -6.8896e-01,
          1.3886e-01, -1.2587e-01,  3.6338e-01, -1.7981e-02, -8.5026e-02],
        [ 4.7122e-01, -7.9934e-02,  4.9980e-01, -1.9960e-01,  5.2866e-01,
          2.0376e-01, -3.6035e-02,  3.5205e-01,  2.2387e-02, -5.8795e-02,
          2.8871e-01, -2.9160e-01,  4.0478e-01,  6.2494e-01, -7.0085e-01,
         -1.3142e-02, -3.3802e-01,  4.0176e-01, -4.9127e-04, -5.7207e-01],
        [ 4.5721e-01,  1.7684e-02,  2.1326e-01, -2.3027e-01,  4.3473e-01,
         -3.9941e-01, -1.6803e-02,  2.1262e-01,  1.8123e-01, -4.2178e-02,
          3.2783e-01, -1.3343e-01,  1.8177e-01,  1.9190e-01, -4.0004e-01,
         -4.4845e-02, -3.3740e-01,  1.4527e-01,  1.1585e-01, -3.4814e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3203, 0.0374, 0.4456, 0.0000, 0.6776,

### `nn.Sequential`
`nn.Sequential` is an ordered container of modules. The data is passed through all the modules in the same order as defined. 

Sequential containers are used to put together a quick network like ``seq_modules``.

In [9]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [10]:
logits

tensor([[-0.2109,  0.0796, -0.1574, -0.0907,  0.1060,  0.1198,  0.2810,  0.1548,
          0.0638,  0.0687],
        [-0.1075,  0.0406, -0.1357, -0.0725, -0.0635,  0.0611,  0.1862,  0.1530,
          0.0315,  0.0607],
        [-0.2372,  0.0536, -0.1293,  0.0025, -0.0137,  0.1142,  0.1570,  0.1503,
          0.1050, -0.0931]], grad_fn=<AddmmBackward0>)

### `nn.Softmax`
The last linear layer of the neural network returns `logits` - raw values in [$\infty, \infty$] which are passed to the `nn.Softmax` module. The logits are scaled to values [0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.



In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
pred_probab

tensor([[0.0769, 0.1029, 0.0812, 0.0867, 0.1056, 0.1071, 0.1258, 0.1109, 0.1012,
         0.1017],
        [0.0880, 0.1020, 0.0855, 0.0911, 0.0919, 0.1041, 0.1180, 0.1141, 0.1011,
         0.1041],
        [0.0774, 0.1036, 0.0863, 0.0984, 0.0968, 0.1100, 0.1149, 0.1141, 0.1090,
         0.0894]], grad_fn=<SoftmaxBackward0>)

## Model Parameters

Many layers inside a neural network have associated weights and biases that are optimized during training, aka Parameterization. Subclassing ``nn.Module`` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0047, -0.0332, -0.0256,  ..., -0.0003,  0.0256, -0.0272],
        [-0.0137,  0.0242, -0.0022,  ..., -0.0173,  0.0005,  0.0186]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0184, -0.0170], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0221,  0.0316,  0.0228,  ..., -0.0397,  0.0351, -0.0035],
        [ 0.0431,  0.0270,  0.0335,  ...,  0.0317,  0.0323, -0.0402]],
       device='mps:0', grad_fn=<Slice

---